In [42]:
from typing import List
import serial
import time
import numpy as np

# Change your usb name here (ex. /dev/ttyUSB0)
serial.Serial()
class Ring:
    def __init__(self, port = 'COM3', baud = 115200, timeout = 0.1):

        self._serial = serial.Serial(port, baud, timeout = timeout)
        self._buf = bytearray(b'')
        self._msg = bytearray(b'')
        self._range_datatype_len = 2
        self._clock_datatype_len = 4

        self.measurements = [[]]

    def close(self):
        self._serial.close()


    def process(self) -> List[List[float]]:

        self.measurements = [[]]

        self._buf += self._serial.read(self._serial.in_waiting)
        
        while ((len(self._buf)) >= 3):
            if ((self._buf[0] == 0xFF) and (self._buf[1] == 0xFF)):
                # btarr[0] and btarr[1] is 0xFF starting bytes
                packet_length = self._buf[2] + 2
                if (len(self._buf) >= packet_length):

                    if (len(self.measurements[0]) == 0):
                        msg_num = 0
                        total_len = len(self._buf)

                    self._msg = self._buf[0:packet_length]
                    self._buf = self._buf[packet_length:]
                    sensors_num = self._msg[3]
                    data_length = self._range_datatype_len * sensors_num
                    
                    if (data_length > packet_length):
                        print("Too long message! Skip measurement.")
                        self._buf = self._buf[data_length:]
                    else:
                        crc = 0
                        for i in range(2, data_length + self._clock_datatype_len + 3):
                            crc = (crc + self._msg[i]) & 0xFF
                        crc = ~(crc & 0xFF) & 0xFF

                        if (crc == (self._msg[-1] & 0xFF)):
                            
                            if (len(self.measurements[0]) == 0):
                                self.measurements = [[0]*(sensors_num + 1) for _ in range((total_len // packet_length))]

                            ranges = [0] * sensors_num

                            for i in range(sensors_num):
                                for j in range(self._range_datatype_len):
                                    ranges[i] += float(self._msg[4+2*i+j] << 8*j)

                            clock = 0.0
                            for i in range(self._clock_datatype_len):
                                clock += float(self._msg[4+data_length+i] << 8*i)

                            for i in range(sensors_num):
                                self.measurements[msg_num][i] = ranges[i]

                            self.measurements[msg_num][-1] = clock
                            msg_num += 1
                            #print("fine message: " + f'Packet length {packet_length}, sensors_num {sensors_num}, ranges = {ranges[0]}, {ranges[1]}; clock_ms = {clock}') 
                     
                        else:
                            print("Corrupted message! Skip measurement.")
                            self._buf = self._buf[(data_length + self._clock_datatype_len + 3):]
                else:
                   return self.measurements

            else:
                self._buf = self._buf[1:]
            
        return self.measurements

    def readMeasurements() -> List[int]:
        pass

In [43]:
right_half_master = Ring(port = 'COM6', baud = 115200, timeout = 0.1)
left_half_slave = Ring(port = 'COM7', baud = 115200, timeout = 0.1)

In [51]:
right_half_master.process()

[[65535.0,
  195.0,
  213.0,
  201.0,
  205.0,
  191.0,
  209.0,
  172.0,
  188.0,
  65535.0,
  185.0,
  191.0,
  28846.0],
 [65535.0,
  198.0,
  206.0,
  200.0,
  207.0,
  198.0,
  207.0,
  172.0,
  189.0,
  65535.0,
  189.0,
  187.0,
  28894.0],
 [65535.0,
  193.0,
  206.0,
  204.0,
  202.0,
  199.0,
  204.0,
  172.0,
  187.0,
  65535.0,
  187.0,
  187.0,
  28941.0],
 [65535.0,
  196.0,
  197.0,
  201.0,
  210.0,
  195.0,
  199.0,
  172.0,
  188.0,
  65535.0,
  184.0,
  188.0,
  28989.0],
 [65535.0,
  191.0,
  207.0,
  204.0,
  203.0,
  197.0,
  204.0,
  172.0,
  190.0,
  65535.0,
  186.0,
  186.0,
  29035.0],
 [65535.0,
  201.0,
  207.0,
  198.0,
  200.0,
  194.0,
  202.0,
  172.0,
  194.0,
  65535.0,
  183.0,
  187.0,
  29082.0],
 [65535.0,
  197.0,
  207.0,
  199.0,
  204.0,
  195.0,
  201.0,
  172.0,
  194.0,
  65535.0,
  186.0,
  189.0,
  29130.0],
 [65535.0,
  197.0,
  209.0,
  201.0,
  206.0,
  197.0,
  198.0,
  172.0,
  187.0,
  65535.0,
  180.0,
  190.0,
  29176.0],
 [65535.

In [49]:
left_half_slave.process()

[[193.0,
  193.0,
  189.0,
  159.0,
  173.0,
  186.0,
  159.0,
  152.0,
  160.0,
  192.0,
  179.0,
  151.0,
  9632.0],
 [196.0,
  193.0,
  191.0,
  159.0,
  174.0,
  186.0,
  163.0,
  157.0,
  154.0,
  196.0,
  178.0,
  151.0,
  9683.0],
 [186.0,
  193.0,
  196.0,
  159.0,
  171.0,
  186.0,
  160.0,
  155.0,
  164.0,
  199.0,
  175.0,
  151.0,
  9732.0],
 [192.0,
  193.0,
  192.0,
  159.0,
  176.0,
  186.0,
  160.0,
  158.0,
  156.0,
  196.0,
  173.0,
  151.0,
  9783.0],
 [190.0,
  193.0,
  192.0,
  159.0,
  170.0,
  183.0,
  161.0,
  155.0,
  158.0,
  195.0,
  177.0,
  151.0,
  9832.0],
 [193.0,
  193.0,
  196.0,
  159.0,
  172.0,
  186.0,
  159.0,
  158.0,
  158.0,
  191.0,
  175.0,
  151.0,
  9882.0],
 [192.0,
  193.0,
  196.0,
  159.0,
  174.0,
  186.0,
  160.0,
  154.0,
  164.0,
  200.0,
  176.0,
  151.0,
  9931.0],
 [183.0,
  193.0,
  194.0,
  159.0,
  175.0,
  186.0,
  162.0,
  155.0,
  166.0,
  197.0,
  176.0,
  151.0,
  9983.0],
 [189.0,
  193.0,
  193.0,
  159.0,
  175.0,
  1

In [31]:
right_half_master.close()

In [32]:
left_half_slave.close()